In [1]:
import numpy as np
from MetaLearner import *
import pickle

In [ ]:
# todo: https://stackoverflow.com/questions/24168569/how-to-add-names-to-a-numpy-array-without-changing-its-dimension

In [ ]:
# This file runs the experiment

In [2]:
file_name = "/Users/arberimbibaj/Documents/Master Thesis ETH/DataSets /Generated/data_synthetic.pkl"
open_file = open(file_name, "rb")
data = pickle.load(open_file)
open_file.close()

In [3]:
n_setups = 2
sample_sizes = [200, 200]
n_runs = 2
learners = [TLearner('rf'), SLearner('rf'), XLearner('rf'), RLearner('rf'), DRLearner('rf'), RALearner('rf'),
            PWLearner('rf'), ULearner('rf'),
            TLearner('lasso'), SLearner('lasso'), XLearner('lasso'), RLearner('lasso'), DRLearner('lasso'),
            RALearner('lasso'), PWLearner('lasso'), ULearner('lasso'),
            TLearner('nn'), SLearner('nn'), XLearner('nn'), RLearner('nn'), DRLearner('nn'), RALearner('nn'),
            PWLearner('nn'), ULearner('nn')]

base_learners = ['rf', 'lasso', 'nn']


In [ ]:
# What we want: For each setting and base learner --> matrix which includes the MSE, columns: Metalearners + SampleSize, rows: Run(1-10)
# --> So we can plot the mean and 1se of the MSE for each Metalearer, each samplesize; giving the setting and baselearner
# --> number of plots: settings*baselearner.

In [4]:
results = []

for i in range(n_setups):
    results.append([])

for i in range(n_setups):
    for baselearner in range(3):
        results[i].append([])


In [ ]:
results[27][2]  # first number indicates the setup, second number indicates the base learner.

In [5]:
results[0]

[[], [], []]

In [ ]:
''''
# helper function to get y, x, w, tau, out of data[][][][]
def get_variables_20(dataset, setup, samplesize, run, train_test):
    y = data[setup][samplesize][run][train_test][:, 0]
    x = data[setup][samplesize][run][train_test][:, 1:21]
    w = data[setup][samplesize][run][train_test][:, 21]
    tau = data[setup][samplesize][run][train_test][:, 22]
    return y, x, w, tau
'''''

In [6]:
# helper function to get y, x, w, tau, out of data[][][][]
def get_variables(dataset, setup, samplesize, run, train_test):
    y = dataset[setup][samplesize][run][train_test][:, 0]
    x = dataset[setup][samplesize][run][train_test][:, 1:26]
    w = dataset[setup][samplesize][run][train_test][:, 26]
    tau = dataset[setup][samplesize][run][train_test][:, 27]
    return y, x, w, tau

In [ ]:
''''
b = 0
m = 0
s = 0
r = 0

for i in range(n_setups):
    b = 0  # restart index
    for base in range(3):  # change to baselearner
        plot_mse = np.empty(shape=(0, 8))  #
        s = 0
        for size in sample_sizes:  # change to sample_sizes
            size_mse = np.empty(shape=(n_runs, 8))  # 10 = number of runs
            for r in range(n_runs):  # run
                mses = np.empty(shape=(1, 8))  # reset
                # set data TODO: make helper function to make y, x, w, tau, out of data[][][][]
                temp_y_train, temp_x_train, temp_w_train, temp_tau_train = get_variables(dataset=data, setup=i,
                                                                                         samplesize=s, run=r,
                                                                                         train_test=0)
                temp_y_test, temp_x_test, temp_w_test, temp_tau_test = get_variables(dataset=data, setup=i, samplesize=s,
                                                                                     run=r, train_test=1)
                s += 1
                m = 0
            for l in learners:  # change to metalearner
                # train and test
                learner = l  # TODO: just abstract, change to metalearner with baselearner here.
                learner.fit(temp_x_train, temp_y_train, temp_w_train)
                predictions = learner.predict(temp_x_test)
                temp_mse = ((predictions - tau_test) ** 2).mean()
                mses[0, m] = temp_mse
                m += 1
            size_mse[r, :] = mses
        b += 1
        np.append(plot_mse, mses)
    results[i][b] = plot_mse







# this should make sense i guess! :)
# TODO: TRY IT OUT TOMORROW!!!!!!, ALSO CHECK IF IT MAKES IT FASTER IF EVERYTHING IS EITHER IN NP OR IN LIST/ARRAY BASE PYTHON.

'''''



In [7]:
#TODO: TRY THIS ONE FIRST!!!

mses = []
b = 0
m = 0
s = 0
r = 0

for i in range(n_setups):
    setup_mse = np.empty(shape=(0, 24))  # baselearner * metalearner
    s = 0
    for size in sample_sizes:  # change to sample_sizes
        size_mse = np.empty(shape=(0, 24))
        for r in range(n_runs):  # run
            mses = np.empty(shape=(1, 24))  # reset
            # set data TODO: make helper function to make y, x, w, tau, out of data[][][][]
            temp_y_train, temp_x_train, temp_w_train, temp_tau_train = get_variables(dataset=data, setup=i,
                                                                                     samplesize=s, run=r,
                                                                                     train_test=0)
            temp_y_test, temp_x_test, temp_w_test, temp_tau_test = get_variables(dataset=data, setup=i, samplesize=s,
                                                                                 run=r, train_test=1)

            m = 0
            for l in learners:  # change to metalearner
                # train and test
                learner = l  # TODO: just abstract, change to metalearner with baselearner here.
                learner.fit(temp_x_train, temp_y_train, temp_w_train)
                predictions = learner.predict(temp_x_test)
                temp_mse = ((predictions - temp_tau_test) ** 2).mean()
                mses[0, m] = temp_mse
                m += 1
            size_mse = np.append(size_mse, mses, axis=0)
            # size_mse[r, :] = mses
        setup_mse = np.append(setup_mse, size_mse, axis=0)
        s += 1
        # np.append(setup_mse, mses)
    results[i][0] = setup_mse[:, 0:8]
    results[i][1] = setup_mse[:, 8:16]  # change
    results[i][2] = setup_mse[:, 16:24]  # change




############# TODO: TODO: lueg nomal, überleg logisch.

# TODO: IT WORKED SOMEHOW, BUT ONLY GET 'MSES' (ONE ROW) VECTOR AND 'SIZE_MSE' (TWO ROWS)
# this should make sense i guess! :)
# TODO: TRY IT OUT TOMORROW!!!!!!, ALSO CHECK IF IT MAKES IT FASTER IF EVERYTHING IS EITHER IN NP OR IN LIST/ARRAY BASE PYTHON.
# TODO: WHILE TRAINING: ConvergenceWarning: Try increasing tol.
# TODO: WHAT DOES PLOT.MSE ???


### TODO: solve : "Input y contains infinity or a value too large for dtype('float64')."
### for: pseudo_outcomes = (y - self.mux_model.predict(x)) / (w - probs)

### IT TOOOK 5M 40S FOR --> SAMPLE SIZES [200, 200], TEST SIZE =50, 2 SETUPS, 2 RUNS LOL

Fitting random forest for mu_0
Fitting random forest for mu_1
Fitting random forest for mu_x
Fitting random forest for mu_0
Fitting random forest for mu_1
Fitting random forest for tau_0
Fitting random forest for tau_1
Fitting random forest for e_x
Fitting random forest for mu_x
Fitting random forest for e_x
Fitting random forest for tau_x
Fitting random forest for mu_0
Fitting random forest for mu_1
Fitting random forest for e_x
Fitting random forest for tau_x
Fitting random forest for mu_0
Fitting random forest for mu_1
Fitting random forest for tau_x
Fitting random forest for e_x
Fitting random forest for tau_x
Fitting random forest for mu_x
Fitting random forest for e_x
Fitting random forest for tau_x
Fitting lasso for mu_0
Fitting lasso for mu_1
Fitting lasso for mu_x
Fitting lasso for mu_0
Fitting lasso for mu_1
Fitting lasso for tau_0
Fitting lasso for tau_1
Fitting lasso for e_x
Fitting lasso for mu_x
Fitting lasso for e_x
Fitting lasso for tau_x
Fitting lasso for mu_0
Fitting 

/Users/arberimbibaj/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.046e+01, tolerance: 5.404e+01
  model = cd_fast.enet_coordinate_descent(
/Users/arberimbibaj/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.261e+01, tolerance: 5.131e+01
  model = cd_fast.enet_coordinate_descent(
/Users/arberimbibaj/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing 

Fitting lasso for mu_x


/Users/arberimbibaj/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.118e+02, tolerance: 9.668e+01
  model = cd_fast.enet_coordinate_descent(
/Users/arberimbibaj/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.161e+02, tolerance: 9.563e+01
  model = cd_fast.enet_coordinate_descent(
/Users/arberimbibaj/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing 

Fitting lasso for mu_0
Fitting lasso for mu_1
Fitting lasso for tau_0
Fitting lasso for tau_1
Fitting lasso for e_x
Fitting lasso for mu_x
Fitting lasso for e_x
Fitting lasso for tau_x
Fitting lasso for mu_0
Fitting lasso for mu_1
Fitting lasso for e_x
Fitting lasso for tau_x
Fitting lasso for mu_0
Fitting lasso for mu_1
Fitting lasso for tau_x
Fitting lasso for e_x
Fitting lasso for tau_x
Fitting lasso for mu_x
Fitting lasso for e_x
Fitting lasso for tau_x
Training neural network for mu_0
Training neural network for mu_1
Training neural network for mu_x
Training NN for mu_0
Training NN for mu_1
Fitting NN for tau_0
Fitting NN for tau_1
Fitting NN for e_x
Training NN for mu_x
Training NN for e_x
Training NN for tau_x
Training NN for mu_0
Training NN for mu_1
Training NN for e_x
Training NN for tau_x
Training NN for mu_0
Training NN for mu_1
Training NN for tau_x
Training NN for e_x
Training NN for tau_x
Training NN for mu_x
Training NN for e_x
Training NN for tau_x
Fitting random fores

/Users/arberimbibaj/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.178e+01, tolerance: 1.915e+01
  model = cd_fast.enet_coordinate_descent(
/Users/arberimbibaj/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.366e+01, tolerance: 1.933e+01
  model = cd_fast.enet_coordinate_descent(


Fitting lasso for mu_1
Fitting lasso for mu_x


/Users/arberimbibaj/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.652e+01, tolerance: 4.230e+01
  model = cd_fast.enet_coordinate_descent(
/Users/arberimbibaj/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.463e+01, tolerance: 4.286e+01
  model = cd_fast.enet_coordinate_descent(
/Users/arberimbibaj/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing 

Fitting lasso for mu_0
Fitting lasso for mu_1
Fitting lasso for tau_0
Fitting lasso for tau_1
Fitting lasso for e_x
Fitting lasso for mu_x
Fitting lasso for e_x
Fitting lasso for tau_x
Fitting lasso for mu_0
Fitting lasso for mu_1
Fitting lasso for e_x
Fitting lasso for tau_x
Fitting lasso for mu_0
Fitting lasso for mu_1
Fitting lasso for tau_x
Fitting lasso for e_x
Fitting lasso for tau_x
Fitting lasso for mu_x
Fitting lasso for e_x
Fitting lasso for tau_x
Training neural network for mu_0
Training neural network for mu_1
Training neural network for mu_x
Training NN for mu_0
Training NN for mu_1
Fitting NN for tau_0
Fitting NN for tau_1
Fitting NN for e_x
Training NN for mu_x
Training NN for e_x
Training NN for tau_x
Training NN for mu_0
Training NN for mu_1
Training NN for e_x
Training NN for tau_x
Training NN for mu_0
Training NN for mu_1
Training NN for tau_x
Training NN for e_x
Training NN for tau_x
Training NN for mu_x
Training NN for e_x
Training NN for tau_x


In [23]:
results[1][2] # NICE :)

array([[ 4.81266021,  0.53981602,  3.4583068 , 10.35876946,  0.71781323,
         5.22196567,  9.86297248,  8.1371202 ],
       [13.11560639,  0.49653672,  5.42221331, 15.73286028,  5.28916795,
         8.15807603, 42.16761745,  6.99697016],
       [18.86482458,  1.69001174,  7.34271276,  3.31280043,  4.39092201,
        15.98248976, 70.83757159,  6.1042887 ],
       [24.84633785,  2.61823956,  9.69743097,  6.28017584,  3.22542507,
        22.44751361, 38.6919919 , 14.95757149]])

In [20]:
mses

array([[5.39904726e+00, 1.62035526e-02, 3.41632611e+00, 8.53723210e+00,
        4.18814312e+00, 3.62821884e+00, 4.86883242e+00, 1.03440314e+01,
        2.33151928e+00, 2.30794607e-01, 7.89986783e-01, 4.76328607e-01,
        7.46131186e-01, 8.60046992e-01, 2.97617878e-01, 5.05113092e-01,
        2.48463379e+01, 2.61823956e+00, 9.69743097e+00, 6.28017584e+00,
        3.22542507e+00, 2.24475136e+01, 3.86919919e+01, 1.49575715e+01]])

In [24]:
size_mse # negative values?
# they do not look good lol, a lot ot zeros, few Nans --> LOL just my fault its the empty matrix :)

array([[8.35256837e+00, 1.90981560e-03, 4.14908013e+00, 4.02424087e+01,
        8.71280249e+00, 8.18541222e+00, 1.13962376e+01, 8.07310756e+01,
        5.38428691e+01, 5.90157811e-01, 1.81801267e+01, 1.53441568e+00,
        3.95228809e+00, 2.14032564e+01, 1.97406846e+01, 1.81519375e+00,
        1.88648246e+01, 1.69001174e+00, 7.34271276e+00, 3.31280043e+00,
        4.39092201e+00, 1.59824898e+01, 7.08375716e+01, 6.10428870e+00],
       [5.39904726e+00, 1.62035526e-02, 3.41632611e+00, 8.53723210e+00,
        4.18814312e+00, 3.62821884e+00, 4.86883242e+00, 1.03440314e+01,
        2.33151928e+00, 2.30794607e-01, 7.89986783e-01, 4.76328607e-01,
        7.46131186e-01, 8.60046992e-01, 2.97617878e-01, 5.05113092e-01,
        2.48463379e+01, 2.61823956e+00, 9.69743097e+00, 6.28017584e+00,
        3.22542507e+00, 2.24475136e+01, 3.86919919e+01, 1.49575715e+01]])

In [25]:
setup_mse

array([[1.96537533e+00, 3.35597974e-03, 9.30893641e-01, 4.27387465e+00,
        1.90672245e+00, 1.92493198e+00, 3.49678582e+00, 4.76913753e+00,
        1.11283226e+00, 0.00000000e+00, 2.73109759e-01, 1.54595104e-01,
        5.41503847e-01, 1.04645801e+00, 2.04329133e+00, 1.58757720e-01,
        4.81266021e+00, 5.39816024e-01, 3.45830680e+00, 1.03587695e+01,
        7.17813233e-01, 5.22196567e+00, 9.86297248e+00, 8.13712020e+00],
       [7.16293784e+00, 2.31834380e-02, 1.43780381e+00, 5.11771216e+01,
        3.31696077e+00, 3.39018449e+00, 2.15809912e+01, 6.48365038e+01,
        2.67661472e+01, 1.62039039e-01, 1.13515856e+01, 2.66536010e-01,
        1.11015144e+00, 1.97699241e+01, 4.15759332e+01, 2.66514897e-01,
        1.31156064e+01, 4.96536722e-01, 5.42221331e+00, 1.57328603e+01,
        5.28916795e+00, 8.15807603e+00, 4.21676174e+01, 6.99697016e+00],
       [8.35256837e+00, 1.90981560e-03, 4.14908013e+00, 4.02424087e+01,
        8.71280249e+00, 8.18541222e+00, 1.13962376e+01, 8.0731

In [ ]:
# nice :)))